**Importing and preparation**

In [ ]:
import td3
import test_utils
import core
import numpy as np
import torch

seed = 42

torch.manual_seed(seed)
np.random.seed(seed)


**Build training data for scenario generator**

In [ ]:
from DiffusionModel.dataprep import compute_technical_indicators , build_tech_dataset , create_lstm_dataset_seq2seq

import numpy as np
import torch
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""
dataset = pd.read_csv("savings/22stocks.csv")
dataset.set_index(['Datetime', 'Ticker'], inplace=True)
dataset.sort_index(inplace=True)
"""
dataset = pd.read_csv('savings/industrial_monthly_data_vw.csv').dropna()



dataset = dataset.set_index(['Date','Industry'])


selected_features = [
    'Monthly_Return'
    ]

window = 12
time_horizon = 12
test_periods = 15
Test_time_horizon = time_horizon * test_periods
pred_len = 1

test_size = Test_time_horizon+window+pred_len - 1
val_size = 200
data_set_end_date = 1




test_dates = dataset.index.get_level_values('Date').unique()[-test_size-data_set_end_date:-data_set_end_date]
val_dates = dataset.index.get_level_values('Date').unique()[-val_size-test_size-data_set_end_date:-test_size-data_set_end_date]
train_dates = dataset.index.get_level_values('Date').unique()[:-val_size-test_size-data_set_end_date]

val_data = dataset.loc[dataset.index.get_level_values('Date').isin(val_dates)]
test_data = dataset.loc[dataset.index.get_level_values('Date').isin(test_dates)]
train_data = dataset.loc[dataset.index.get_level_values('Date').isin(train_dates)]





X_train, y_train = create_lstm_dataset_seq2seq(train_data,
                                   selected_features,
                                   target_col='Next_Monthly_Return',
                                   window_size=window,
                                   pred_len=pred_len)

X_test,  y_test  = create_lstm_dataset_seq2seq(test_data,
                                   selected_features,
                                   target_col='Next_Monthly_Return',
                                   window_size=window,
                                   pred_len=pred_len)

X_val , y_val  = create_lstm_dataset_seq2seq(val_data,
                                   selected_features,
                                   target_col='Next_Monthly_Return',
                                   window_size=window,
                                   pred_len=pred_len)


scale = 1
scale_c = 100
X_train = X_train*scale
X_test = X_test*scale
X_val = X_val*scale
y_train = y_train*scale
y_test = y_test*scale
y_val = y_val*scale


# Convert to torch tensors
X_train_t = torch.from_numpy(X_train).float().to(device)   # (n_train, W, n_assets*num_features)
y_train_t = torch.from_numpy(y_train).float().to(device)  # (n_train, W, n_assets)
X_test_t  = torch.from_numpy(X_test).float().to(device)    # (100, W, n_assets*num_features)
y_test_t  = torch.from_numpy(y_test).float().to(device)    # (100, W, n_assets)
X_val_t = torch.from_numpy(X_val).float().to(device)
y_val_t = torch.from_numpy(y_val).float().to(device)

# Wrap in TensorDataset + DataLoader
train_ds = TensorDataset(X_train_t, y_train_t)
test_ds  = TensorDataset(X_test_t,  y_test_t)
val_ds = TensorDataset(X_val_t, y_val_t)


train_loader = DataLoader(train_ds, batch_size=64, shuffle=True,  drop_last=True)
test_loader  = DataLoader(test_ds,  batch_size=64, shuffle=False, drop_last=False)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False, drop_last=False)


**Initialize the diffusion model for scenario generator**

In [ ]:
from DiffusionModel.core import TrainingNetwork



input_size = X_train_t.shape[2]
print("Input dimension:", input_size)  # number of features per stock

target_dim = y_train_t.shape[3]  # number of assets
print("Target dimension (= # of assets):", target_dim)  # number of assets

num_layers = 2
num_cells= 20
cell_type = "LSTM"

model = TrainingNetwork(input_size=input_size , num_cells= num_cells , num_layers= num_layers , cell_type=cell_type ,dropout_rate = 0.1,target_dim=target_dim,
diff_steps = 500,
prediction_length = pred_len,
beta_min = 0.01,
beta_end = 20,
scale_c = 1,
md_type = 'vpsde'
)

model.to(device)
#One forward pass to initialize the model
with torch.no_grad():
    rnn_output, _ = model.rnn(X_train_t[-1:,-1:,:])
    cond = rnn_output[-1:,-1:,:] # Forward pass to initialize the model
    tstep = torch.rand((1,)).to(device)
    model.model(X_train_t[-1:,-pred_len:,:], tstep, cond)

save_path_diffusion = f'savings/models/diffusion_model_industrial_exp_dataset{data_set_end_date}_final.pth'


**Train the diffusion model**

In [ ]:
from DiffusionModel.trainer import fit , TrainCfg , evaluate

cfg = TrainCfg(
    optim_name="AdamW",
    scheduler="cosine",
    lr=1e-3,
    num_epochs=1000,
    use_ema=True,
    warmup_steps=10,
    amp=True,
    patience=500,
)
model.to(device)

model.train()

fit(model, train_loader, val_loader, device, cfg, save_path_diffusion)



/content/drive/.shortcut-targets-by-id/1w6OSQ_9ZH9r3830ldxRe1TGWfVlxxXWo/RL_real_0625_ind_index_value_weighted/DiffusionModel/trainer.py:143: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=cfg.amp)
Epoch 1:   0%|          | 0/12 [00:00<?, ?it/s]/content/drive/.shortcut-targets-by-id/1w6OSQ_9ZH9r3830ldxRe1TGWfVlxxXWo/RL_real_0625_ind_index_value_weighted/DiffusionModel/trainer.py:158: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=cfg.amp):
Epoch 1:   8%|▊         | 1/12 [00:00<00:07,  1.55it/s, loss=1.12, lr=0.001]/content/drive/.shortcut-targets-by-id/1w6OSQ_9ZH9r3830ldxRe1TGWfVlxxXWo/RL_real_0625_ind_index_value_weighted/DiffusionModel/trainer.py:158: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  wit

[  1/1000]  train=0.3797  val=0.5984  lr=1.00e-03  t=0.0m


[  2/1000]  train=0.0717  val=0.1738  lr=9.99e-04  t=0.0m


[  3/1000]  train=0.0445  val=0.0555  lr=9.97e-04  t=0.0m


Epoch 4:   0%|          | 0/12 [00:00<?, ?it/s, loss=0.0356, lr=0.000997]/content/drive/.shortcut-targets-by-id/1w6OSQ_9ZH9r3830ldxRe1TGWfVlxxXWo/RL_real_0625_ind_index_value_weighted/DiffusionModel/trainer.py:158: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=cfg.amp):


...

Epoch 765:  50%|█████     | 6/12 [00:00<00:00, 19.51it/s, loss=0.0205, lr=7.37e-5]/content/drive/.shortcut-targets-by-id/1w6OSQ_9ZH9r3830ldxRe1TGWfVlxxXWo/RL_real_0625_ind_index_value_weighted/DiffusionModel/trainer.py:158: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=cfg.amp):


[765/1000]  train=0.0178  val=0.0193  lr=7.78e-05  t=7.5m


[766/1000]  train=0.0161  val=0.0249  lr=8.82e-05  t=7.5m


Epoch 767:  42%|████▏     | 5/12 [00:00<00:00, 15.69it/s, loss=0.022, lr=9.27e-5]/content/drive/.shortcut-targets-by-id/1w6OSQ_9ZH9r3830ldxRe1TGWfVlxxXWo/RL_real_0625_ind_index_value_weighted/DiffusionModel/trainer.py:158: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=cfg.amp):


[767/1000]  train=0.0185  val=0.0249  lr=9.92e-05  t=7.5m
Early stopping


**Prepare the scenario generator**

In [7]:
from DiffusionModel.config import get_ddpm_config

#from core import sample_offline, sample_online


config = get_ddpm_config()
config['sampling']['corrector'] = 'langevin'
config['sampling']['predictor'] = 'reverse_diffusion'

**Training the RL agent**

In [8]:
num_assets = model.target_dim
fea_dim = model.feature_dim

retrain_frequency = 90


poolsize = 500

obs_dim = fea_dim + 2
act_dim = num_assets
algo_list = ['GenTD3', 'EW', 'GenMarkowitz', 'HistMarkowitz']
type_list = ['Return']

ac = core.MLPActorCritic_TD3(obs_dim, act_dim)
ac.to(device)

for j in range(Test_time_horizon//retrain_frequency):
    print(f"Retrain: {j+1}/{Test_time_horizon//retrain_frequency}")
    scenario_pool = core.get_scenario_pool(num_assets, fea_dim, time_horizon+1, poolsize, type='generative',
                                history=X_test_t[j*retrain_frequency:,:,:], generative_model=model, config=config, scale = scale_c, load_file = save_path_diffusion)



    regularization = 0.5 if j == 0 else 0.3

    ac, epoch_list= td3.gen_td3(ac, num_assets, fea_dim, time_horizon+1, scenario_pool ,
                          seed=seed,
                          steps_per_epoch=50, epochs=200,
                          pool_size = poolsize,
                          scale_t = 10,
                          regularization1 = regularization,
                          regularization2 = regularization,
                          regularization_time = 0.5,
                          polyak=0.99, pi_lr=3e-5, q_lr=3e-5, pi_lr_min = 1e-6, q_lr_min = 1e-6,
                          batch_size=16, start_steps=50,
                          update_after=50, update_every=50, policy_delay = 2)

    save_path_rl = f'savings/models/td3_industrial_exp_ol_dataset{data_set_end_date}_retrain{j+1}_final.pth'
    core.save_td3_checkpoint(ac.pi, ac.q1, filename = save_path_rl, episode= 100)


Retrain: 1/2
Loading diffusion model from savings/models/diffusion_model_industrial_exp_dataset1_final.pth
Sampling timestep 1/12
Sampling timestep 2/12
Sampling timestep 3/12
Sampling timestep 4/12
Sampling timestep 5/12
Sampling timestep 6/12
Sampling timestep 7/12
Sampling timestep 8/12
Sampling timestep 9/12
Sampling timestep 10/12
Sampling timestep 11/12
Sampling timestep 12/12
Logging data to /tmp/experiments/1752338174/progress.txt

Number of parameters: 	 pi: 18762, 	 q1: 18817, 	 q2: 18817

Checkpoint saved to checkpoints/td3_ep2.pth
-------------------------------------
|          LossPi |           0.115 |
|           LossQ |        1.95e+03 |
|            Time |             5.3 |
-------------------------------------
Step 99: pi_lr=0.000030, q_lr=0.000030
Checkpoint saved to checkpoints/td3_ep3.pth
-------------------------------------
|          LossPi |           0.302 |
|           LossQ |        1.83e+03 |
|            Time |            10.7 |
--------------------------

In [ ]:
scale_c

100

**Prepare the predictions for the GenMarkowitz portfolio**

In [ ]:
prediction_returns, _ = model.sample_online(config, Test_time_horizon, 500, X_test_t, compare = False)



#save prediction_returns
np.save(f'savings/variables/genmkw_returns_industrial_exp_dataset{data_set_end_date}_final.npy', prediction_returns)

Sampling timestep 1/180
Sampling timestep 2/180
Sampling timestep 3/180
Sampling timestep 4/180
Sampling timestep 5/180
Sampling timestep 6/180
Sampling timestep 7/180
Sampling timestep 8/180
Sampling timestep 9/180
Sampling timestep 10/180
Sampling timestep 11/180
Sampling timestep 12/180
Sampling timestep 13/180
Sampling timestep 14/180
Sampling timestep 15/180
Sampling timestep 16/180
Sampling timestep 17/180
Sampling timestep 18/180
Sampling timestep 19/180
Sampling timestep 20/180
Sampling timestep 21/180
Sampling timestep 22/180
Sampling timestep 23/180
Sampling timestep 24/180
Sampling timestep 25/180
Sampling timestep 26/180
Sampling timestep 27/180
Sampling timestep 28/180
Sampling timestep 29/180
Sampling timestep 30/180
Sampling timestep 31/180
Sampling timestep 32/180
Sampling timestep 33/180
Sampling timestep 34/180
Sampling timestep 35/180
Sampling timestep 36/180
Sampling timestep 37/180
Sampling timestep 38/180
Sampling timestep 39/180
Sampling timestep 40/180
Sampling 